# Web Scraping: The https://www.reuters.com

In [176]:
from datetime import datetime
import numpy as np
import pandas as pd


from bs4 import BeautifulSoup
from selenium import webdriver
import time

import requests
import re


### Obtain list of news from the coverpage

URL definition:

In [177]:
# url definition
url = "https://www.reuters.com"

List of news:

In [178]:
# Request
r1 = requests.get(url)
print(r1.status_code)

# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html5lib')

# News identification

coverpage_news = soup1.find_all('h3', class_='article-heading')
#coverpage_news=coverpage_news+coverpage_news2
print (len(coverpage_news))

coverpage_news

200
20


[<h3 class="article-heading"><a href="/article/us-yes-bank-stocks/yes-bank-ceo-says-lender-very-stable-shares-surge-idUSKBN1WI07B?il=0">Yes Bank CEO says lender 'very stable'; shares surge</a>
 	</h3>,
 <h3 class="article-heading"><a href="/article/us-thomas-cook-grp-investment-passengers/uk-to-fly-back-another-5000-people-on-thursday-after-thomas-cook-collapse-idUSKBN1WI0FA?il=0">UK to fly back another 5,000 people on Thursday after Thomas Cook collapse</a>
 	</h3>,
 <h3 class="article-heading"><a href="/article/us-southeast-asia-internet/southeast-asias-internet-economy-to-hit-300-billion-by-2025-report-idUSKBN1WI07X?il=0">Southeast Asia's internet economy to hit $300 billion by 2025: report</a>
 	</h3>,
 <h3 class="article-heading"><a href="/article/us-australia-economy-salesfigures/australian-car-sales-slump-again-in-september-as-banks-tighten-lending-idUSKBN1WI0DU?il=0">Australian car sales slump again in September as banks tighten lending</a>
 	</h3>,
 <h3 class="article-heading"

Now we have a list in which every element is a news article:

In [179]:
coverpage_news[0]


<h3 class="article-heading"><a href="/article/us-yes-bank-stocks/yes-bank-ceo-says-lender-very-stable-shares-surge-idUSKBN1WI07B?il=0">Yes Bank CEO says lender 'very stable'; shares surge</a>
	</h3>

In [180]:
n=0
link = coverpage_news[n].find('a')['href']
title = coverpage_news[n].get_text()
article = requests.get(url+link)
article_content = article.content
soup_article = BeautifulSoup(article_content, 'html5lib')


In [181]:
title

"Yes Bank CEO says lender 'very stable'; shares surge\n\t"

In [182]:
body = soup_article.find_all('p')

In [183]:
body

[<p class="BylineBar_reading-time" style="color:undefined">3 Min Read</p>,
 <p>MUMBAI (Reuters) - Private-sector lender Yes Bank’s (<span id="”symbol_YESB.NS_0”"><a href="//www.reuters.com/companies/YESB.NS">YESB.NS</a></span>) Chief Executive Officer Ravneet Gill assured investors on Thursday that the bank remains on solid financial footing, sending its stock as much as 25% higher.  </p>,
 <p>His remarks come after the stock plunged nearly 23% on Tuesday as fraud allegations against a housing finance company that Yes Bank has exposure to, spooked investors. A stake sale by its promoters also weighed on sentiment. </p>,
 <p>Two of the bank’s promoter companies, led by former CEO Rana Kapoor and his family, said on Thursday they cut stakes over the last two weeks to reduce debt in their own companies, further seeking to allay investor concerns. </p>,
 <p>The recent price drop does not reflect the bank’s performance, its CEO said during a conference call, adding the operating metrics and

In [184]:
len(body)

17

In [185]:
x = soup_article.find_all('p')

In [186]:
len(x)

17

In [187]:
x[0].get_text()

'3 Min Read'

### Let's extract the text from the articles:

First, we'll define the number of articles we want:

In [188]:
number_of_articles = len(coverpage_news)-1
now = datetime.now()

In [189]:
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []
scrapedTime=[]
postedDate=[]
for n in np.arange(0, number_of_articles):
        
    # Getting the link of the article
    link = coverpage_news[n].find('a')['href']
    print (link)
    list_links.append(link)
    posteddate=re.findall(r'/(\d{4})/(\d{1,2})/(\d{1,2})/', link)
    #posteddate=str(posteddate)
    if posteddate:
        posteddate=','.join(posteddate[0]) 
        posteddate=posteddate.replace(',','/')
    print (posteddate)
    postedDate.append(posteddate)
    
    # Getting the title
    title = coverpage_news[n].get_text()
    list_titles.append(title)
    
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(url+link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    x = soup_article.find_all('p')
    scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    scrapedTime.append(scraped)
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)

/article/us-yes-bank-stocks/yes-bank-ceo-says-lender-very-stable-shares-surge-idUSKBN1WI07B?il=0
[]
/article/us-thomas-cook-grp-investment-passengers/uk-to-fly-back-another-5000-people-on-thursday-after-thomas-cook-collapse-idUSKBN1WI0FA?il=0
[]
/article/us-southeast-asia-internet/southeast-asias-internet-economy-to-hit-300-billion-by-2025-report-idUSKBN1WI07X?il=0
[]
/article/us-australia-economy-salesfigures/australian-car-sales-slump-again-in-september-as-banks-tighten-lending-idUSKBN1WI0DU?il=0
[]
/article/us-hongkong-protest-markets/about-4-billion-hong-kong-deposits-may-have-left-for-singapore-as-protests-rage-goldman-idUSKBN1WI0CR?il=0
[]
/article/us-hongkong-protests/hong-kong-protesters-rampage-across-city-as-police-call-for-curfew-idUSKBN1WI02F?il=0
[]
/article/us-northkorea-missiles/north-korea-says-it-successfully-tested-new-submarine-launched-ballistic-missile-idUSKBN1WH2GS?il=0
[]
/article/us-usa-trump-whistleblower/u-s-diplomat-at-center-of-trump-ukraine-affair-to-meet-w

In [171]:
df=pd.DataFrame(columns= ["STOCK ID", "URL", "PUBLISH TIME", "EDITED TIME", "SCRAPED TIME", "TITLE", "CONTENT" ])
df['URL']=list_links
df['PUBLISH TIME']=postedDate
df['SCRAPED TIME']=scrapedTime
df['TITLE']=list_titles
df['CONTENT']=news_contents
df

,STOCK ID,URL,PUBLISH TIME,EDITED TIME,SCRAPED TIME,TITLE,CONTENT
0,NaN,/article/us-northkorea-missiles/north-korea-sa...,[],NaN,02/10/2019 18:09:00,North Korea says successfully tested new subma...,2 Min Read SEOUL (Reuters) - North Korea said ...
1,NaN,/article/us-tesla-safety-nhtsa/troubles-with-t...,[],NaN,02/10/2019 18:09:00,Troubles with Tesla's automated parking summon...,4 Min Read WASHINGTON/BENGALURU (Reuters) - U....
2,NaN,/article/us-usa-election-guns/sensing-shift-de...,[],NaN,02/10/2019 18:09:00,"Sensing shift, Democratic presidential candida...",6 Min Read LAS VEGAS (Reuters) - Democratic pr...
3,NaN,/article/us-wto-aircraft-drink-food/u-s-hits-s...,[],NaN,02/10/2019 18:09:00,"U.S. hits Scotch whisky, Italian cheese, Frenc...",2 Min Read WASHINGTON (Reuters) - The Trump ad...
4,NaN,/article/us-wto-aircraft/u-s-set-to-widen-trad...,[],NaN,02/10/2019 18:09:00,U.S. set to widen trade war with new tariffs o...,6 Min Read LONDON/BRUSSELS/WASHINGTON (Reuters...
5,NaN,/article/us-usa-trump-whistleblower-pompeo/tru...,[],NaN,02/10/2019 18:09:00,Trump slams impeachment probe as hoax as Democ...,7 Min Read WASHINGTON (Reuters) - President Do...
6,NaN,/article/us-usa-election-sanders-medicare/sand...,[],NaN,02/10/2019 18:09:00,Sanders presidential campaign pivots health sc...,3 Min Read WASHINGTON (Reuters) - Bernie Sande...
7,NaN,/article/us-wto-aircraft-usa-tariffs/u-s-to-hi...,[],NaN,02/10/2019 18:09:00,U.S. to hit EU with tariffs of 10% on aircraft...,1 Min Read WASHINGTON (Reuters) - The United S...
8,NaN,/article/us-usa-stocks/wall-street-tumbles-as-...,[],NaN,02/10/2019 18:09:00,Wall Street tumbles as trade war threatens U.S...,5 Min Read (Reuters) - Wall Street’s main inde...
9,NaN,/article/us-saudi-khashoggi-un/saudi-prince-se...,[],NaN,02/10/2019 18:09:00,Saudi prince seeks to dodge blame for Khashogg...,4 Min Read GENEVA (Reuters) - Saudi Arabia’s c...


In [172]:
st=pd.DataFrame(pd.read_csv('c:/data/CS/tickers_list.csv'))
st['Issuer Name'][0].split(',')
issuerName=[]
stockabbre=[]
for i in range(len(st['Issuer Name'])):
    issuername=st['Issuer Name'][i].split(',')
    issuerName.append(issuername[0])
print (issuerName)
stockabbre=st["Symbol"]
stockabbre=list(stockabbre)
stockabbre.append('APPL')
print (stockabbre)
    

['ADOMANI', 'AdvisorShares Trust', 'Allied Esports Entertainment', 'Alta Mesa Resources', 'Alta Mesa Resources', 'BIQI International Holdings Corporation', 'China Lending Corporation', 'China TechFaith Wireless Communication Technology Limited', 'ChinaCache International Holdings Ltd.', 'Cray Inc', "Del Frisco's Restaurant Group", 'Diversicare Healthcare Services Inc.', 'Emmaus Life Sciences', 'Emmaus Life Sciences', 'Finisar Corporation', "Fred's", 'Gladstone Capital Corporation', 'Hunter Maritime Acquisition Corp.', 'Hunter Maritime Acquisition Corp.', 'Hunter Maritime Acquisition Corp.', 'Ivy NextShares', 'Ivy NextShares', 'Ivy NextShares', 'Peak Resorts', 'Repay Holdings Corporation', 'Shutterfly', 'USA Technologies', 'USA Technologies', 'Xynomic Pharmaceuticals Holdings', 'Xynomic Pharmaceuticals Holdings', 'Xynomic Pharmaceuticals Holdings', 'Yangtze River Port and Logistics Limited ']
['ADOM', 'BKCH', 'AESEW', 'AMR', 'AMRWW', 'BIQI', 'CLDC', 'CNTF', 'CCIH', 'CRAY', 'DFRG', 'DVCR

In [173]:
for i in range(len(df['CONTENT'])):
    for j in range(len(st['Symbol'])):
        if st['Symbol'][j] in df['CONTENT'][i] or issuerName[j] in df['CONTENT']:
            df['STOCK ID'][i]=st['Symbol'][j]
df['STOCK ID'].describe()

count     0
unique    0
Name: STOCK ID, dtype: int64

In [175]:
df.to_csv("C:/data/CS/reuters_SCRAPE.csv")